## PDI - Phuture DeFi Index - Rebalancing Data March 2024

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyProd
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
date = '2024-03-01'
min_mcap =  2.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["decentralized-finance-defi"]

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyProd(
                                date,
                                1,
                                'ethereum',
                                '0x632806BF5c8f062932Dd121244c9fbe7becb8B48',
                                'pdi_benchmark_data',
                                'pdi_liquidities',
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project must have a listing on Defi Llama.
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. 
    1. Projects that have circular feedback characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
   <br>
1. **Pricing requirmeents**
    1. Token should have a manipulation resistant price feed in the form of a Chainlink price feed or a Uniswap V3 TWAP oracle with a cardinality of at least 300 and a minimum liquidity level of $1mm.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have in aggregate at least \$5mm of onchain liquidity across Uniswap v2, Uniswap v3, Sushiswap, Balancer v1, Balancer v2, Bancor v2 and Bancor v3.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    1. The token must have shown consistent DeFi liquidity on Ethereum.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    'ethereum',
    df_to_remove=[stablecoins,ctokens,atokens],
    add_category_assets=['liquity'],
    remove_category_assets=['the-graph','staked-ether','injective-protocol','chainlink','gnosis']
    )
liquidity

0xd7c49cee7e9188cca6ad8ff264c1da2e69d4cf3b ethereum
yearn-finance did not meet the average three month liquidity threshold
pendle did not meet the average three month liquidity threshold
reserve-rights-token did not meet the average three month liquidity threshold
pancakeswap-token did not meet the average three month liquidity threshold
0x did not meet the average three month liquidity threshold
gnosis did not meet the average three month liquidity threshold
band-protocol did not meet the average three month liquidity threshold
uma did not meet the average three month liquidity threshold
Dropping dydx because a price feed is unavailable


,ethereum,best slippage,best slippage chain,verify homechain slippage
id,,,,
uniswap,-0.005032,-0.005032,ethereum,True
rocket-pool,-0.007754,-0.007754,ethereum,True
maker,-0.007921,-0.007921,ethereum,True
lido-dao,-0.008916,-0.008916,ethereum,True
convex-finance,-0.010764,-0.010764,ethereum,True
balancer,-0.013108,-0.013108,ethereum,True
aave,-0.020127,-0.020127,ethereum,True
1inch,-0.026944,-0.026944,ethereum,True
compound-governance-token,-0.028550,-0.028550,ethereum,True


## Show Results

In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
uniswap,UNI,11881283712,0.300000,77,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,ethereum
lido-dao,LDO,2944101024,0.145489,38,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
maker,MKR,1907849942,0.094281,25,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,ethereum
aave,AAVE,1773823352,0.087657,23,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,ethereum
havven,SNX,1391653144,0.068772,18,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,ethereum
ribbon-finance,RBN,1084797829,0.053608,14,0x6123b0049f904d730db3c36a31167d9d4121fa6b,ethereum
curve-dao-token,CRV,930367577,0.045976,12,0xd533a949740bb3306d119cc777fa900ba034cd52,ethereum
rocket-pool,RPL,698806006,0.034533,9,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,ethereum
1inch,1INCH,696195847,0.034404,8,0x111111111117dc0aa78b770fa6a738034120c302,ethereum
frax-share,FXS,605648318,0.029929,7,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,ethereum


## Contract Output

In [5]:
methodology_class.output_for_contract()

0x111111111117dc0aa78b770fa6a738034120c302,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,0x560363bda52bc6a44ca6c8c9b4a5fadbda32fa60,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x6123b0049f904d730db3c36a31167d9d4121fa6b,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,0x6dea81c8171d0ba574754ef6f8b412f2ed88c54d,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0xba100000625a3754423978a60c9317c58a424e3d,0xc00e94cb662c3520282e6f5717214004a7f26888,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,0xd533a949740bb3306d119cc777fa900ba034cd52
8,77,7,5,3,38,14,4,2,23,25,3,7,18,9,12
